In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from ganutilities import manage_batch_2c
from createganmodels import new_gan_discriminator, new_gan_generator

In [15]:
# input variables for training

input_ticks = 3200
noise_dim = 100
batch_size = 50
learning_rate = 1e-3
data_path = 'fillsplit3200.pickle'
starting_epoch = 25

In [18]:
# You can either create new models, or load already trained ones to continue training

#discriminator = new_gan_discriminator(input_ticks)
#generator = new_gan_generator(input_ticks)

gen_path = './bigepochs/gene25'
generator = tf.keras.models.load_model(gen_path)

disc_path = './bigepochs/dise25'
discriminator = tf.keras.models.load_model(disc_path)



In [4]:
# Uses cross entropy loss (as recommended usually for GANs)
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# Discriminator loss is given by the loss from real_output and fake_output
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

# Discriminator successfully guessing the generator is fake is higher loss for the generator

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# Use Adam optimizer (as recommended for many neural nets)
generator_optimizer = tf.keras.optimizers.Adam(learning_rate)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate)

In [19]:
# trains gan for input number of epochs
def train_dcgan(epochs):
    # uses batch manager to get new shuffled batches every epoch
    batch_manager = manage_batch_2c(data_path, batch_size)
    for i in range(epochs):
        batches = batch_manager.new_epoch()
        for batch in batches:
            formatted_batch = batch
            
            # generated latent space for generator
            noise = tf.random.normal([batch_size, noise_dim])
            
            # generates sequences, and gets generator and discriminator loss
            with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
                generated_sequences = generator(noise, training=True)

                real_output = discriminator(formatted_batch, training=True)
                fake_output = discriminator(generated_sequences, training=True)

                gen_loss = generator_loss(fake_output)
                disc_loss = discriminator_loss(real_output, fake_output)
                print(gen_loss, disc_loss)
            
            # applying gradients
            gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
            gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

            generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
            discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
        
        
        generator.save("./bigepochs/gene" + str(i+starting_epoch+1))
        discriminator.save("./bigepochs/dise" + str(i+starting_epoch+1))
        print("finished epoch!", i+starting_epoch + 1)


In [20]:
# trains gan for 10 epochs
train_dcgan(10)

tf.Tensor(1.173255, shape=(), dtype=float32) tf.Tensor(7.2446275, shape=(), dtype=float32)
tf.Tensor(346.95258, shape=(), dtype=float32) tf.Tensor(357.7815, shape=(), dtype=float32)
tf.Tensor(83.99507, shape=(), dtype=float32) tf.Tensor(91.46596, shape=(), dtype=float32)
tf.Tensor(0.0, shape=(), dtype=float32) tf.Tensor(107.936554, shape=(), dtype=float32)
tf.Tensor(1.2369725, shape=(), dtype=float32) tf.Tensor(9.884946, shape=(), dtype=float32)
tf.Tensor(274.7884, shape=(), dtype=float32) tf.Tensor(245.25587, shape=(), dtype=float32)
tf.Tensor(207.21594, shape=(), dtype=float32) tf.Tensor(161.24785, shape=(), dtype=float32)


INFO:tensorflow:Assets written to: ./bigepochs/gene26/assets


INFO:tensorflow:Assets written to: ./bigepochs/gene26/assets


INFO:tensorflow:Assets written to: ./bigepochs/dise26/assets


INFO:tensorflow:Assets written to: ./bigepochs/dise26/assets


...

INFO:tensorflow:Assets written to: ./bigepochs/gene35/assets


INFO:tensorflow:Assets written to: ./bigepochs/gene35/assets


INFO:tensorflow:Assets written to: ./bigepochs/dise35/assets


INFO:tensorflow:Assets written to: ./bigepochs/dise35/assets


finished epoch! 35
